In [10]:
import cv2
from ultralytics import YOLO
import traceback
import numpy as np
import time

# First Attempt at Running Model From Camera

In [ ]:
model = YOLO('yolov8m-pose.pt')
results = model(source=0, show=True, conf=.3, save=False)

This code has a few problems. First there is no way to analyze individual frames and secondly, the video can't be stopped until you reset the kernel.

# Keypoints and Angles Between Joints

I want to get each joint from each individual frame and calculate the angles for each joint. Before I had no way of getting data on a single frame, however, if I use openCV to capture from my camera I can pass in the frames individually. After running the YOLOv8 model on a single frame it returns its results. From this you are able to get the 17 keypoints making up the pose. Each of these points represent the pixel coordinates of each join. The points are all stored in an array in the following order:

0. nose
1. left eye 
2. right eye
3. left ear
4. right ear
5. left shoulder
6. right shoulder
7. left elbow
8. right elbow
9. left wrist
10. right wrist
11. left waist
12. right waist
13. left knee
14. right knee
15. left ankle
16. right ankle

## Calculating Angles

In [2]:
def calc_angle(point_A, point_B, point_C):
    # Calculate vectors from B to A and B to C
    vector_AB = point_A - point_B
    vector_BC = point_C - point_B

    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_AB, vector_BC)

    # Calculate the magnitudes (lengths) of the vectors
    magnitude_AB = np.linalg.norm(vector_AB)
    magnitude_BC = np.linalg.norm(vector_BC)

    # Calculate the cosine of the angle θ using the dot product formula
    cosine_theta = dot_product / (magnitude_AB * magnitude_BC)

    # Calculate the angle θ in radians using the arccosine function
    angle_radians = np.arccos(cosine_theta)

    # Convert the angle from radians to degrees
    angle_degrees = np.degrees(angle_radians)

    return (angle_degrees, angle_radians)

## Annotating Angles on Capture

In [ ]:
# Initialize FPS variables
frame_count = 0
fps = -1
start_time = time.time()

# Angle indices
joint_angle_indices = {
    "left_elbow": [5, 7, 9],
    "right_elbow": [6, 8, 10],
    "left_shoulder": [11, 5, 7],
    "right_shoulder": [12, 6, 8],
    "left_waist": [5, 11, 13],
    "right_waist": [6, 12, 14],
    "left_knee": [11, 13, 15],
    "right_knee": [12, 14, 16]
}

# Load the YOLOv8 model
model = YOLO('yolov8n-pose.pt')

# Start video capture
cap = cv2.VideoCapture(0)

try:
    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:

            # Calculate and display FPS
            frame_count += 1
            if frame_count >= 10:  # Calculate FPS every 10 frames
                end_time = time.time()
                elapsed_time = end_time - start_time
                fps = frame_count / elapsed_time
                frame_count = 0
                start_time = end_time

            # Run YOLOv8 inference on the frame
            results = model(source=frame, conf=.3)

            # Get keypoints from results
            pixel_keypoints = results[0].keypoints.xy[0]

            # Calculate and annotate the angle of each joints
            if (len(pixel_keypoints) != 0):
                # Visualize the pose results on the frame
                annotated_frame = results[0].plot()

                for joint_name in joint_angle_indices:
                    indices = joint_angle_indices[joint_name]
                    img_points = [np.array(pixel_keypoints[i]) for i in indices]
                    cv2.putText(
                        annotated_frame,
                        str(calc_angle(*img_points)[0]),
                        tuple(map(int, img_points[1])),
                        cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1
                    )

                # Display FPS
                cv2.putText(annotated_frame, 'FPS: ' + str(fps), (0,25), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1)

                # Display the annotated frame
                cv2.imshow("YOLOv8 Inference", annotated_frame)
            else:
                # Display FPS
                cv2.putText(frame, 'FPS: ' + str(fps), (0,0), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)

                # Show frame
                cv2.imshow("", frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
except Exception as err:
    traceback.print_exc()
finally:
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()



# MediaPipe

[Download Models](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models)
<br>
[Docs](https://developers.google.com/mediapipe/api/solutions/python/mp)

In [4]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import cv2
import traceback
import time

In [2]:
def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [5]:
# Initialize FPS variables
frame_count = 0
fps = -1
start_time = time.time()

# Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker_full.task')
options = vision.PoseLandmarkerOptions(base_options=base_options)
model = vision.PoseLandmarker.create_from_options(options)

# Start video capture
cap = cv2.VideoCapture(0)

try:
    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Calculate and display FPS
            frame_count += 1
            if frame_count >= 10:  # Calculate FPS every 10 frames
                end_time = time.time()
                elapsed_time = end_time - start_time
                fps = frame_count / elapsed_time
                frame_count = 0
                start_time = end_time

            # Convert the OpenCV frame to RGB format (mediapipe expects RGB)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=np.asarray(rgb_frame))

            # Detect pose landmarks from the input image.
            results = model.detect(image)


            if results.pose_landmarks:
                # Draw the pose landmarks on the frame
                annotated_image = draw_landmarks_on_image(image.numpy_view(), results)

                # Display FPS
                cv2.putText(annotated_image, 'FPS: ' + str(fps), (0,25), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1)

                cv2.imshow('', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
            else:
                # Display FPS
                cv2.putText(frame, 'FPS: ' + str(fps), (0,25), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1)

                cv2.imshow('', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
except Exception as err:
    traceback.print_exc()
finally:
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()



## Hand Landmarks

In [ ]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import time
import traceback

In [1]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [2]:
# Initialize FPS variables
frame_count = 0
fps = -1
start_time = time.time()

# Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
model = vision.HandLandmarker.create_from_options(options)

# Start video capture
cap = cv2.VideoCapture(0)

try:
    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Calculate and display FPS
            frame_count += 1
            if frame_count >= 10:  # Calculate FPS every 10 frames
                end_time = time.time()
                elapsed_time = end_time - start_time
                fps = frame_count / elapsed_time
                frame_count = 0
                start_time = end_time

            # Convert the OpenCV frame to RGB format (mediapipe expects RGB)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=np.asarray(rgb_frame))

            # Detect pose landmarks from the input image.
            results = model.detect(image)

            if results.hand_landmarks:
                # Draw the pose landmarks on the frame
                annotated_image = draw_landmarks_on_image(image.numpy_view(), results)

                # Display FPS
                cv2.putText(annotated_image, 'FPS: ' + str(fps), (0,25), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1)

                cv2.imshow('', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
            else:
                # Display FPS
                cv2.putText(frame, 'FPS: ' + str(fps), (0,25), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 1)

                cv2.imshow('', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
except Exception as err:
    traceback.print_exc()
finally:
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

